## 1. XGBoost

In [346]:
import numpy as np
# dataset = 'data-nci'
dataset = 'data-oneil'
# dataset = 'data-drugcomb-fi'
# dataset = 'data-DrugCombDB'
fold_n = 5
xTr = np.load('./' + dataset + '/form_data/xTr' + str(fold_n) + '.npy')
yTr = np.load('./' + dataset + '/form_data/yTr' + str(fold_n) + '.npy')
drugTr =  np.load('./' + dataset + '/form_data/drugTr' + str(fold_n) + '.npy')

xTe = np.load('./' + dataset + '/form_data/xTe' + str(fold_n) + '.npy')
yTe = np.load('./' + dataset + '/form_data/yTe' + str(fold_n) + '.npy')
drugTe = np.load('./' + dataset + '/form_data/drugTe' + str(fold_n) + '.npy')

In [347]:
xdrugTr = np.hstack((xTr, drugTr))
xdrugTe = np.hstack((xTe, drugTe))
print(xdrugTr.shape)
print(yTr.shape)
print(xdrugTe.shape)
print(yTe.shape)

(1176, 8110)
(1176, 1)
(294, 8110)
(294, 1)


In [348]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# Load your data


# Prepare DMatrix
dtrain = xgb.DMatrix(xdrugTr, label=yTr)
dtest = xgb.DMatrix(xdrugTe, label=yTe)

# Set parameters for regression
params = {
    'max_depth': 2,
    'eta': 0.1,
    'objective': 'reg:squarederror',
    'nthread': 1,
    'eval_metric': 'rmse'
}
num_round = 20

# Train the model
bst = xgb.train(params, dtrain, num_round)

# Predict
preds = bst.predict(dtest)

# Evaluate with RMSE
mse = mean_squared_error(yTe, preds)
print("MSE: %.2f" % mse)
rmse = np.sqrt(mean_squared_error(yTe, preds))
print("RMSE: %.2f" % rmse)


RMSE: 12.74


In [349]:
# convert list of lists to list
yTe = np.array(yTe).flatten()
print(yTe.tolist())

[-17.2817979072755, 5.79128728304933, 3.7921116228684086, 0.51700222845629, 1.0966146270020711, -6.613686761665703, -39.873530292237, -35.0272315856836, 0.2475632146873464, 5.59963570142504, 5.29419053806362, 6.457932781589695, 6.4251548431743295, -19.7885249955934, -2.4097304318056905, 7.543407488617369, 1.83593808226401, -1.5082203021212124, 5.99108205682079, -18.1797235112709, -12.840088322870097, -2.23616712442702, -4.662511414401839, 3.41802282280863, -12.0320822507053, -25.9801803981754, -5.97419023328963, 15.4186986077963, 4.480465220149085, -13.2363849677504, -9.362863686069195, 4.128157012788561, 2.26767773086615, -24.1634597090208, 10.013204486477417, -31.934070451053, -22.9515728231767, -11.219410776078313, 6.00089213826483, -28.9638748432599, 3.11049964318654, -1.073248118887358, -10.249148139079496, -13.14009076535719, 0.7364260970712593, -44.9949251983702, -5.159519294761907, 1.869241020829697, -55.54568697318505, 0.3065104335081801, -10.386374558578522, -3.03452459021913

In [350]:
import pandas as pd
# Create DataFrame from yTe and preds
preds = np.array(preds).flatten()
data = pd.DataFrame({
    'Actual': yTe.tolist(),
    'Predicted': preds.tolist()
})

data.to_csv('./ml_results/xgboost/' + dataset + '/result_' + str(fold_n) + '.csv', index=False, header=True)

# Calculate Pearson correlation
correlation = data.corr(method='pearson').iloc[0, 1]
print("Pearson correlation: %.6f" % correlation)

Pearson correlation: 0.693886


In [351]:
nci_pearson_corr_list = [0.636280, 0.607164, 0.621125, 0.562870, 0.547934]
oneil_pearson_corr_list = [0.678435, 0.633401, 0.638970, 0.660978, 0.693886]
drugcomb_fi_pearson_corr_list = [0.555848, 0.558369, 0.530912, 0.557552, 0.553214]
drugcomb_db_pearson_corr_list = [0.515926, 0.496845, 0.517539, 0.522624, 0.540706]

## 2. Random Forest

In [1039]:
import numpy as np
# dataset = 'data-nci'
# dataset = 'data-oneil'
# dataset = 'data-drugcomb-fi'
dataset = 'data-DrugCombDB'
fold_n = 5
xTr = np.load('./' + dataset + '/form_data/xTr' + str(fold_n) + '.npy')
yTr = np.load('./' + dataset + '/form_data/yTr' + str(fold_n) + '.npy')
drugTr =  np.load('./' + dataset + '/form_data/drugTr' + str(fold_n) + '.npy')

xTe = np.load('./' + dataset + '/form_data/xTe' + str(fold_n) + '.npy')
yTe = np.load('./' + dataset + '/form_data/yTe' + str(fold_n) + '.npy')
drugTe = np.load('./' + dataset + '/form_data/drugTe' + str(fold_n) + '.npy')

In [1040]:
xdrugTr = np.hstack((xTr, drugTr))
xdrugTe = np.hstack((xTe, drugTe))
print(xdrugTr.shape)
print(yTr.shape)
print(xdrugTe.shape)
print(yTe.shape)

(17116, 9594)
(17116, 1)
(4283, 9594)
(4283, 1)


In [1041]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error

# Assuming xTr and yTr are already defined
# Create the Random Forest Regressor model
model = RandomForestRegressor(n_estimators=5)

# Fit the model on the training data
model.fit(xdrugTr, yTr)

# Predicting the training set itself to see how well we fit the training data
predictions = model.predict(xdrugTe)

C:\Users\hemingzhang\AppData\Local\Temp\ipykernel_257372\2098612624.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(xdrugTr, yTr)


In [1042]:
# convert list of lists to list
yTe = np.array(yTe).flatten()
print(yTe.tolist())

[-61.34, 8.94, -1.47, -4.731, 11.0, 0.49, 6.78, -64.7, 6.93, -16.1, -1.97, -17.5, -74.98, -2.97, -5.3, 4.19, 5.46, -4.99, -1.43, -13.62, -28.16, -3.77, -11.535, -4.12, 0.0, -1.67, 8.0725, -20.45, 3.32, -0.18, -3.06, -57.29, -11.48, -8.71, 3.63, -0.48, -27.58, -15.27, -7.28, -1.89, 10.19, -1.75, -3.04, 10.94, 4.86, -19.21, -38.72, -0.96, 3.67, -0.26, -35.44, -1.68, -1.55, -24.94, -10.84, -7.39, -15.25, -2.18, 10.58, -5.35, -13.15, -33.91, -0.81, -0.25, 3.12, -4.86, 4.24, -18.72, -1.63, 0.71, 3.43, 6.11, -16.36, -3.38, -9.11, -1.43, -1.79, -2.39, -28.59, 8.27, -40.27, 5.53, -15.83, -39.23, -3.3, 6.13, -7.73, 5.47, -25.37, -3.304, -15.44, -33.52, -30.84, -1.35, -3.11, 3.46, 10.53, 0.32, -2.55, -7.4, -0.22, -7.21, 5.18, -80.89, 5.73, -2.6, 8.92, -2.06, -64.69, -13.61, -8.5, -3.2325, -5.56, -1.33, -11.84, -7.17, -2.9, 12.34, -10.11, 10.68, -43.56, 3.24, -0.24, -29.02, -1.21, 1.41, -11.24, -5.24, 1.41, -0.4, -2.32, -34.89, -0.75, -1.83, 5.14, -6.51, -51.14, -4.52, -51.0, -8.03, 1.72, -27.567

In [1043]:
print(len(predictions))

4283


In [1044]:
import pandas as pd
# Create DataFrame from yTe and preds
preds = np.array(predictions).flatten()
data = pd.DataFrame({
    'Actual': yTe.tolist(),
    'Predicted': preds.tolist()
})

data.to_csv('./ml_results/rf/' + dataset + '/result_' + str(fold_n) + '.csv', index=False, header=True)

# Calculate Pearson correlation
correlation = data.corr(method='pearson').iloc[0, 1]
print("Pearson correlation: %.6f" % correlation)

Pearson correlation: 0.670877


In [1045]:
nci_pearson_corr_list = [0.669306, 0.635840, 0.652682, 0.569410, 0.590657]
oneil_pearson_corr_list = [0.678316, 0.685238, 0.665807, 0.708294, 0.677736]
drugcomb_fi_pearson_corr_list = [0.652296, 0.652927, 0.648258, 0.652175, 0.658323]
drugcomb_db_pearson_corr_list = [0.649099, 0.629549, 0.644015, 0.647977, 0.670877]